# Analysis of  RPM basal-organoid-derived allograft tumours
## Ireland et al 2024 BioRxiv (Accepted at Nature, 2025)
## Updated 05.2025
### Related to Fig. 2e (Final Fig. 1j) and Extended Data Fig. 4e

In [ ]:
#Import other relevant packages
import numpy as np
import pandas as pd
from matplotlib import rcParams
import os
import scanpy as sc

import matplotlib as mpl
import matplotlib.pyplot as plt

#For nice color schemes
import cmocean

#For barplots
import seaborn as sns

#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

#Import scVI
import scvi
from scvi.model.utils import mde

scvi.settings.verbosity = 40

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
# Read 
os.chdir('/work/asi16')

## 1. Read in RPM Allograft "CellTagged pre-Cre"

In [ ]:
RPM_allo=sc.read_10x_mtx('MedGenome_FASTQ_123123/RPM_TBO_Allo/042024_RPM_TBO_Allo_CustomCount/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)

## 2. Read in RPM Allograft ("Allo3") "CellTagged post-Cre"

In [ ]:
RPM_allo3=sc.read_10x_mtx('042225_RPM_TBO_CellTagPostCre/042225_RPMTBOAllo_CTpostCre_1/outs/filtered_feature_bc_matrix', var_names='gene_symbols', cache=True)

In [ ]:
# Add metadata to RPMA and RPM
RPM_allo.obs['Genotype'] = 'RPM'
RPM_allo.obs['GenoCT'] = 'RPM_CTpreCre'
RPM_allo.obs['Model'] = 'Allograft'
RPM_allo.obs['Cre'] = 'CT_pre-Cre'
RPM_allo.obs['UnID'] = 'RPM_Allo_New'
RPM_allo.obs['Batch'] = 'RPM_Allo_New'

RPM_allo3.obs['Genotype'] = 'RPM'
RPM_allo3.obs['GenoCT'] = 'RPM_CTpostCre'
RPM_allo3.obs['Model'] = 'Allograft'
RPM_allo3.obs['Cre'] = 'CT_post-Cre'
RPM_allo3.obs['UnID'] = 'RPM_Allo3'
RPM_allo3.obs['Batch'] = 'RPM_CTpostCre'


## 3. Concatenate allograft adata objects to one adata object

In [ ]:
# Concatenate datasets, first without the no cre old RPM sample...
adata= RPM_allo.concatenate([RPM_allo3], 
                              index_unique=None, join="outer")

In [ ]:
adata.obs_names_make_unique()

In [ ]:
adata.obs.groupby(["UnID"]).apply(len)

## 4. Perform QC

In [ ]:
#QC filtering
adata.var['mito'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

sc.pp.filter_cells(adata, min_genes=200)
#sc.pp.filter_genes(orgs_all, min_cells=3)

adata.var['mito'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mito'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mito'],
             jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mito')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
#Filter data by slicing anndata object
adata = adata[adata.obs.n_genes_by_counts < 9000, :]
adata = adata[adata.obs.n_genes_by_counts > 500, :]
adata = adata[adata.obs.total_counts > 2000, :]
adata = adata[adata.obs.pct_counts_mito < 15, :]

In [ ]:
adata.obs.groupby(["UnID"]).apply(len)

In [ ]:
adata.obs.groupby(["Batch"]).apply(len)

In [ ]:
adata.obs.groupby(["GenoCT"]).apply(len)

In [ ]:
adata.obs.groupby(["Genotype"]).apply(len)

## 5. ID HVGs and generate SCVI model for clustering with batch correction

In [ ]:
#Prep for HVG and scvi
#log1p the data
adata.obs["log1p_total_counts"] = np.log1p(adata.obs["total_counts"])
#Create layers
adata.layers["counts"] = adata.X.copy()
adata.layers['norm'] = adata.X.copy(); sc.pp.normalize_total(adata, target_sum=1e4, layer="norm")

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3"
)

## 6. Set up and train model (scvi)

In [ ]:
adata.var['mean_'] = np.array(adata.X.mean(0))[0]
adata.var['frac_zero'] = 1 - np.array((adata.X > 0).sum(0))[0] / adata.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata.var.mean_, adata.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata.varm['df_poisson']= df_poisson

adata_query = adata[:, is_hvg].copy()
print(adata_query)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata.obsm["X_scVI_1.1"] = latent

## 7. Perform leiden clustering

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata, use_rep="X_scVI_1.1")
sc.tl.umap(adata, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata, key_added="leiden_scVI_1.1", resolution=1)

## 8. Visualize clustering, QC metrics, and identify cell types

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="leiden_scVI_1.1", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="Genotype", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata, color="GenoCT", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)

#Additional QC bar graphs
adata.obs['cluster'] = adata.obs["leiden_scVI_1.1"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata.obs, x="cluster", y="pct_counts_mito", ax=ax)


In [ ]:
# Dot plot key cell type markers
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers

sc.set_figure_params(scanpy=True, fontsize=20) 
sc.pl.dotplot(
    adata,figsize=[28,10],
    var_names=more_types,
    groupby='leiden_scVI_1.1',
    use_raw=False,
    layer="norm",show=False,
    color_map="cmo.dense", var_group_rotation=35,standard_scale='var',
    save=False)

In [ ]:
#feature plots
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Fabp1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Marco","Mrc1","Ly75","Adgre1",
            "Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4","Clec4a1", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93"] #Endothelial

sc.pl.umap(
    adata,
    color=more_types,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,s=30,
    frameon=False,
    vmax="p99.5",
    layer="norm",
    save=False
)

In [ ]:
adata.write_h5ad("042725_RPMTBOAllos_OriginalandAllo4_5kHVG.h5ad")

In [ ]:
# Subsample by cluster here to maintain heterogeneity/representation of rare cell types/cell fates.

# Set the number of cells you want to sample per cluster
n_cells_per_cluster = 300  # Change this as needed

# Create a list to hold the downsampled indices
downsampled_indices = []

# Loop over each unique Leiden cluster label
for cluster in adata.obs['leiden_scVI_1.1'].unique():
    # Get the indices of cells in this cluster
    cluster_cells = adata.obs[adata.obs['leiden_scVI_1.1'] == cluster].index
    
    # If there are more than n_cells_per_cluster cells, sample randomly
    if len(cluster_cells) > n_cells_per_cluster:
        sampled_cells = np.random.choice(cluster_cells, size=n_cells_per_cluster, replace=False)
    else:
        sampled_cells = cluster_cells  # Don't downsample if the cluster has fewer cells
    
    # Append the sampled indices to the list
    downsampled_indices.extend(sampled_cells)

# Create a new AnnData object with just the downsampled cells
adata_downsampled = adata[downsampled_indices, :].copy()

# Print the new object size
print(adata_downsampled)


In [ ]:
# Based on gene expression and QC metrics, remove non-tumor cell clusters and low QC clusters
# 24-endothelial 15-immune 21-qa 12-qc 11-qc 0-qc 3-qc 16-qc

bad_clust=['24','15','21','12','11','0','3','16']

#Filter out bad clusters
to_keep=(~adata_downsampled.obs['leiden_scVI_1.1'].isin(bad_clust))

#Copy over to new anndata object
adata2 = adata_downsampled[to_keep].copy()

### From here, continue iterating through runs of scvi modeling until no clear low quality cell clusters or non-tumor cells are observed.
### Start back up at "set up and train scvi model" and run through subsetting out "bad clusters".
### Each time clusters are removed, model is run again to recluster.

## ITERATION 2

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata2,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3"
)

In [ ]:
adata2.var['mean_'] = np.array(adata2.X.mean(0))[0]
adata2.var['frac_zero'] = 1 - np.array((adata2.X > 0).sum(0))[0] / adata2.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata2.var.mean_, adata2.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata2, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata2.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata2.varm['df_poisson']= df_poisson

adata_query = adata2[:, is_hvg].copy()
print(adata_query)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata2.obsm["X_scVI_1.2"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata2, use_rep="X_scVI_1.2")
sc.tl.umap(adata2, min_dist=1)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata2, key_added="leiden_scVI_1.2", resolution=0.5)


In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata2, color="leiden_scVI_1.2", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata2, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata2, color="GenoCT", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)

#Additional QC bar graphs
adata2.obs['cluster'] = adata2.obs["leiden_scVI_1.2"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata2.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata2.obs, x="cluster", y="pct_counts_mito", ax=ax)


In [ ]:
more_types=["Col14a1", "Acta2","Myh11","Tagln","Mustn1", #fibroblast
              "Lpl","Lipa","Pparg","Plin2","Ear1","Spp1", #lipofibroblast/osteoblastic
              "Ptprc","Mertk","Mrc1","Ly75","Adgre1","Itgax","Cd68","Csf1r","Mafb","Msr1","Arg1","Adgre4", #Macs/Myeloid
              "Cx3cr1","Itgam","Cd14", #Monocytes
              "S100a9","S100a8","Mmp9","Csf3r","Cxcr2","Ly6g", #Neuts
              "Batf3","Xcr1","Clec9a","Ccl17","Ccl22", #DC
              "Cd3d","Cd3e","Cd3g","Cd28","Cd8a","Cd4","Foxp3", # Tcell
              "Gzma","Ncr1","Gzmb", #NK
              "Fcmr","Cd19","Fcer2a","Pax5","Cd22","Cd79b","Cd79a", #B cells
              "Slamf7", "Prdm1", #Plasma
              "Mcam","Pecam1","Icam2","Cd36","Cd93", #Endothelial
    "Pdpn","Cav1","Cav2","Hopx","Timp3","Sema3f","Serpine1", #AT1
              "Abca3","Muc1","Sftpa1","Sftpb","Sftpd","Scd1", #AT2
              "Scgb1a1","Cyp2f2","Scgb3a2", "Scgb3a1","Lypd2",#Club
              "Muc5ac","Muc5b", # Goblet
              "Tubb4a","Foxa3","Foxj1","Rfx2","Rfx3","Trp73", #Ciliated
              'Krt5', 'Krt17','Krt15','Trp63','Id1','Icam1','Epas1','Aqp3','Sfn','Perp','Fxyd3','Sdc1','Gstm2','F3','Abi3bp','Adh7', # Basal
              'Bex2','Ascl1','Meis2','Hes6','Hoxb5','Foxa2','Sox4','Rora','Isl1','Id4', 'Neurod1','Neurod4','Nhlh1','Nhlh2',#NE/neuronal
              'Pou2f3','Trpm5','Ascl2','Ehf',
              'Lrmp','Gng13','Ltc4s','Alox5ap','Avil','Alox5','Atp2a3','Plk2', #tuft
              "Cftr","Ascl3", 'Stap1','Atp6v1c2','Pparg','Rasd1','Slc12a2', #ionocyte
              "Gja1","Nkx2-1","Epcam", # Lung lineage
              'Yap1','Wwtr1','Sox2','Cd44','Hes1', # Stem-like
             "Venus","fLuc", "Top2a","Mki67",
            'GFP.CDS','CellTag.UTR'] # Tumor markers
sc.set_figure_params(scanpy=True, fontsize=20) 
sc.pl.dotplot(
    adata2,figsize=[28,10],
    var_names=more_types,
    groupby='leiden_scVI_1.2',
    use_raw=False,
    layer="norm",show=False,
    color_map="cmo.dense", var_group_rotation=35,standard_scale='var',
    save=False)

In [ ]:
#QC UMAPs
sc.pl.umap(
    adata2,
    color=["n_genes_by_counts", "total_counts", "pct_counts_mito", "log1p_total_counts"],
    cmap="cubehelix_r",
    s=3,
    ncols=2,
)

In [ ]:
adata2.write_h5ad("042525_RPM_TBOAllo_OriginalandAllo3_1.2_HVG5k_subsampledbycluster_300cells.h5ad")

In [ ]:
#feature plots
genes = ['leiden_scVI_1.2', 'Trp63', 'Sox2',
        'Mki67', 'Top2a', 'Ascl1','Atoh1',
        'Neurod1', 'Pou2f3','Ascl2','Gng13', 'Cftr',
        'Ptprc', 'Pecam1', 'Acta2', 'GFP.CDS','CellTag.UTR','fLuc']

sc.pl.umap(
    adata2,
    color=genes,
    use_raw=False,
    legend_loc= "on data",
    color_map="cmo.dense",
    ncols=4,s=60,
    frameon=False,
    layer="norm",
    save=False
)

In [ ]:
# Exclude cluster 10, as it is a fibroblast cluster
bad_clust=['10']

#Filter out bad clusters
to_keep=(~adata2.obs['leiden_scVI_1.2'].isin(bad_clust))

#Copy over to new anndata object
adata3 = adata2[to_keep].copy()

# ITERATION 3 
## Final iteration for Fig. 2e (Final Fig. 1j) RPM allograft UMAP

In [ ]:
#HVG via Scanpy
#Note here that if you run with a batch_key with few cells, will get b'reciprocal condition number error
sc.pp.highly_variable_genes(
    adata3,n_top_genes=5000,
    subset=False,
    layer="counts",
    flavor="seurat_v3"
)

In [ ]:
# Show the genes that were selected as highly variable
hvg_genes = adata3.var_names[adata3.var['highly_variable']]
hvg_genes.to_series().to_csv('highly_variable_genes.csv', index=False)

In [ ]:
adata3.var['mean_'] = np.array(adata3.X.mean(0))[0]
adata3.var['frac_zero'] = 1 - np.array((adata3.X > 0).sum(0))[0] / adata3.shape[0]

fig, ax = plt.subplots(figsize=(9,6))
ax.scatter(adata3.var.mean_, adata3.var.frac_zero, s=1)
ax.set_xscale("log")

In [ ]:
#Calculate Poisson gene selection
df_poisson = scvi.data.poisson_gene_selection(
    adata3, n_top_genes=5000, batch_key="Batch", inplace=False
)

df_poisson[df_poisson.highly_variable].sort_values('prob_zero_enrichment_rank')

pd.crosstab(df_poisson.highly_variable, adata2.var.highly_variable)

is_hvg = df_poisson.highly_variable

adata3.varm['df_poisson']= df_poisson

adata_query = adata3[:, is_hvg].copy()
print(adata_query)

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata_query,
    layer="counts",
    batch_key='Batch',
    continuous_covariate_keys=["pct_counts_mito"]
)

model = scvi.model.SCVI(adata_query)

In [ ]:
#Train and run scvi

#Training parameters
train_kwargs = dict(
    early_stopping=True,
    early_stopping_patience=20,
    enable_model_summary=True,
    enable_progress_bar=True,
    enable_checkpointing=True,
    max_epochs=500
)

#Train and run model
#Be sure GPU is enabled to run this
model.train(**train_kwargs)

In [ ]:
#Fit model to data
#Get latent representation of model to apply to UMAP
latent = model.get_latent_representation()

adata3.obsm["X_scVI_1.3"] = latent

In [ ]:
#Calculate neighbors using scVI model input
sc.pp.neighbors(adata3, use_rep="X_scVI_1.3")
sc.tl.umap(adata3, min_dist=0.5)

#Run leiden clustering based on neighbors
sc.tl.leiden(adata3, key_added="leiden_scVI_1.3", resolution=1)


## Plot UMAPs of Leiden clusters and individual sample distribution
## Visualize QC metrics

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata3, color="leiden_scVI_1.3", legend_loc="on data", legend_fontsize='large',ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))
sc.pl.umap(adata3, color="UnID", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)
fig, ax = plt.subplots(figsize=(8, 6))

sc.pl.umap(adata3, color="GenoCT", legend_loc="right margin", ax=ax, s=30, frameon=False, save=False)

#Additional QC bar graphs
adata3.obs['cluster'] = adata3.obs["leiden_scVI_1.3"].copy()

#Plot Log1p total counts
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata3.obs, x="cluster", y="log1p_total_counts", ax=ax)

#Plot Pct counts mito
fig, ax = plt.subplots(figsize=(20,6))
sns.boxenplot(data=adata3.obs, x="cluster", y="pct_counts_mito", ax=ax)


## Calculate cluster markers for Supplementary Tables and fate identification

In [ ]:
# Find cluster markers for each leiden cluster to aid filtering
sc.tl.rank_genes_groups(adata3, 'leiden_scVI_1.3', method='wilcoxon', layer='norm', use_raw=False)
pd.DataFrame(adata3.uns['rank_genes_groups']['names']).head(50)


In [ ]:
# Extract the ranked genes (names) for all clusters
marker_genes = pd.DataFrame(adata3.uns['rank_genes_groups']['names'])

# Save to CSV
marker_genes.to_csv('042725_adata3_subbycluster_RPMoriginalandAllo3_cluster_marker_genes.csv', index=False)

## Visualize marker genes for Fig. 2e (Final Fig. 1j)

In [ ]:
# Visualize marker genes
genes = ['Ascl1','Neurod1','Pou2f3','Atoh1',
         'Yap1','Trp63']

plt.rcParams['figure.figsize'] = [10, 8]
sc.set_figure_params(scanpy=True, fontsize=20)

# Plot without showing immediately
g = sc.pl.umap(
    adata3,
    color=genes,
    use_raw=False,
    legend_loc="right margin",
    color_map="cmo.dense",
    ncols=6,
    frameon=False,
    vmax="p99.9",
    layer="norm",
    s=50,
    show=False,
    return_fig=True
)

# Make gene names (titles) larger and italic
for ax in g.axes:
    ax.set_title(ax.get_title(), fontsize=40, fontstyle='italic')

plt.tight_layout()
plt.show()

In [ ]:
adata3=sc.read_h5ad("042725_RPM_TBOAllos_OriginalandAllo3_adata3_5kHVG_subsamplebycluster.h5ad")

## DotPlot of key cell type/cell fate markers in Extended Data Fig. 4e

In [ ]:
# Dot plot for Extended Data Fig. 4e
more_types=["Trp63","Krt5","Krt15","Krt17",#basal
            "Ascl1", "Syp","Chga","Insm1","Chgb","Myt1","Sez6","Foxa2","Mycl", #NE
              "Neurod1","Nhlh1","Nhlh2","Neurod2", #neuronal
            'Atoh1','Ush2a','Lhx3','Rasd2','Pou4f3', # Atoh1/Inner Ear
            'Pou2f3','Trpm5','Ascl2','Lrmp','Gng13','Avil','Alox5','Atp2a3', #tuft
              "Cftr",'Foxi1', "Ascl3", 'Stap1','Pparg', #ionocyte
              'Yap1','Wwtr1','Sox2','Cd44','Hes1',"Vim", # Stem-like/#Mesenchymal/Yap
             "Top2a","Mki67", "Ube2c","Aspm", #Proliferation
            'Myc',"fLuc"] #tumor markers

# Dot plot for Extended Data Fig. 4e
sc.set_figure_params(scanpy=True, fontsize=17)  # controls default font sizes

dotplot = sc.pl.dotplot(
    adata3,
    var_names=more_types,
    groupby='leiden_scVI_1.3',
    use_raw=False,
    layer="norm",
    figsize=[15, 5],
    show=False,
    color_map="cmo.dense",
    var_group_rotation=35,
    smallest_dot=10,
    dendrogram=True
)

# Increase x-axis label size
plt.xticks(fontsize=20)  # adjust size as needed
plt.yticks(fontsize=16)
plt.tight_layout()
plt.show()

## End of analysis in Scanpy, move to Seurat in R for calculation of gene signatures and additional plot generation/visualization. To do this, convert resulting h5ad anndata object from this script as a Seurat object in R.